In [42]:
import pandas as pd
import mysql.connector
from sqlalchemy import create_engine
import urllib.parse
from datetime import datetime
import pytz
import requests
import numpy as np

# Table of Content
* [Data Extraction](#Data-Extraction)
* [Data Filtering](#Data-Filtering)
* [Customer Reach](#Customer-Reach)
    * [Feature 1: cc_id](#Feature-1:-cc_id)
    * [Feature 2: num_cc](#Feature-2:-num_cc)
    * [Feature 3: channel](#Feature-3:-channel)
    * [Feature 4: trial_money](#Feature-4:-trial_money)
    * [Dependent Variable: trial_completed](#Dependent-Variable:-trial_completed)
* [Finalization](#Finalization)

# Data Extraction

In [44]:
# Fetch data from MySQL (attribute_info)
query1 = "SELECT attriid as 'attri_id', attriname as 'attri_name', attriname_en as 'attri_name_en', pid FROM ehailuo_attribute_info_glv"
attribute = pd.read_sql(query1, engine)
attribute

,attri_id,attri_name,attri_name_en,pid
0,1,小学生,Elementary Students,0.0
1,2,中学生,Junior/Senior Students,0.0
2,3,大学生,Undergraduates,0.0
3,4,销售/客服/采购,Sales/Customer Service/Purchase,0.0
4,5,IT/通信/电子,IT/Communication/E-commerce,0.0
...,...,...,...,...
2752,3187,抖音直播,None,3172.0
2753,3188,CR30天停课学员回访,,NaN
2754,3189,青少儿应试,Young Learners Test Preparation,568.0
2755,3190,小红书投放,None,3172.0


In [45]:
# Fetch data from MySQL (member)
query2 = "SELECT userid as user_id, ccid as cc_id FROM ehailuo_member"
cc = pd.read_sql(query2, engine)
cc

,user_id,cc_id
0,16410781,0.0
1,16448486,0.0
2,16531800,0.0
3,16410859,59.0
4,16410861,59.0
...,...,...
237804,16553294,101915.0
237805,16553296,101915.0
237806,16553307,101915.0
237807,16553310,101915.0


In [46]:
# Fetch data from MySQL (member_detail)
query3 = "SELECT userid as user_id, area as channel FROM ehailuo_member_detail"
channel = pd.read_sql(query3, engine)
channel

,user_id,channel
0,100001,335.0
1,113040,0.0
2,113043,0.0
3,113050,671.0
4,113086,671.0
...,...,...
242720,16583131,512.0
242721,16583132,3120.0
242722,16583133,512.0
242723,16583134,512.0


In [47]:
# Fetch data from MySQL (ccchange_log)
query4 = "SELECT studentid as user_id, operatorid as operator_id, prevccid as prev_cc_id, transtoccid as trans_to_cc_id, transtime as trans_time FROM ehailuo_ccchange_log"
cc_change = pd.read_sql(query4, engine)
cc_change

,user_id,operator_id,prev_cc_id,trans_to_cc_id,trans_time
0,161710,145,111,143,1502356636
1,161709,204,0,204,1502357188
2,161716,204,212,205,1502357254
3,161717,204,212,204,1502357282
4,100001,-2,206,206,0
...,...,...,...,...,...
1675752,16583128,54,0,54,1690457162
1675753,16583129,167,0,167,1690459000
1675754,16583132,154,0,154,1690460921
1675755,158503,59,0,154,1690462058


In [48]:
# Fetch data from MySQL (product_buy_glv)
query5 = "SELECT userid as user_id, productid as product_id, productname as product_name, producttype as buy_type, productmoney as product_money FROM ehailuo_product_buy_glv"
order = pd.read_sql(query5, engine)
order.tail()

,user_id,product_id,product_name,buy_type,product_money
231204,16499394.0,17.0,海螺币,3185.0,0.0
231205,16517179.0,17.0,海螺币,3185.0,0.0
231206,16581580.0,17.0,海螺币,3185.0,0.0
231207,16562491.0,41.0,取消约课卡,1151.0,0.0
231208,16583135.0,26.0,体验卡,749.0,0.0


In [49]:
# Find the class reservation history
query6 = "SELECT classorderid as class_order_id, studentid as user_id, classorderstatus as class_order_status, paytype as pay_type FROM ehailuo_class_orderlist_glv"
class_reservation = pd.read_sql(query6, engine)
class_reservation

,class_order_id,user_id,class_order_status,pay_type
0,1,401,3,1
1,2,402,1,1
2,3,401,0,1
3,4,401,2,1
4,5,401,1,1
...,...,...,...,...
1359719,1363903,16557469,1,25
1359720,1363904,141620,1,21
1359721,1363905,16576479,1,25
1359722,1363906,16568079,1,25


In [50]:
# Read the CSV file into a DataFrame
member_demographics = pd.read_csv('member_demographics.csv')
member_demographics

,user_id,age,gender,job,after_policy
0,131638,420.0,1,0,0
1,131639,417.0,0,0,0
2,131640,413.0,2,0,0
3,131641,416.0,0,0,0
4,131642,420.0,0,0,0
...,...,...,...,...,...
158085,16579599,412.0,2,16,1
158086,16579600,417.0,0,16,1
158087,16579601,412.0,2,1,1
158088,16579602,419.0,0,16,1


# Data Filtering

Filter the data according to the user_id in the member_demographics table.

In [51]:
# Get a list of user_id of members for analysis
member_analysis_list = member_demographics['user_id'].tolist()

In [52]:
# Filter the table according to the member_analysis_list
customer_reach_raw = pd.merge(cc, channel, on='user_id', how='inner')
customer_reach = customer_reach_raw[customer_reach_raw['user_id'].isin(member_analysis_list)]
customer_reach

,user_id,cc_id,channel
0,16410781,0.0,1663.0
1,16448486,0.0,288.0
2,16531800,0.0,3009.0
5,16532204,100.0,288.0
6,16398016,197.0,672.0
...,...,...,...
237744,16558015,647.0,3109.0
237746,16569340,647.0,3156.0
237747,16578239,647.0,3156.0
237748,16578345,647.0,3156.0


In [53]:
customer_reach.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 158090 entries, 0 to 237749
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   user_id  158090 non-null  int64  
 1   cc_id    158089 non-null  float64
 2   channel  158025 non-null  float64
dtypes: float64(2), int64(1)
memory usage: 4.8 MB


# Customer Reach

#### Feature 1: cc_id

The customer service representative (CC) plays a crucial role in shaping the customer experience. They are responsible for various tasks, such as assigning trial classes, suggesting teaching materials, and recommending teachers based on customer preferences. Therefore, the decisions made by CCs have a significant impact on customers' overall experience with the company.

In [54]:
customer_reach['cc_id'].replace(np.nan, 0, inplace=True)

/var/folders/b9/g48b_66n63n3rwlbb7dhxm0r0000gn/T/ipykernel_38401/2698610288.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  customer_reach['cc_id'].replace(np.nan, 0, inplace=True)


In [55]:
customer_reach['cc_id'].value_counts()

0.0      136871
167.0      3822
134.0      2505
54.0       1477
319.0      1360
          ...  
539.0         1
555.0         1
244.0         1
84.0          1
214.0         1
Name: cc_id, Length: 177, dtype: int64

The majority of customers have a CC ID of 0. This occurs when customers decline to continue their experience or remain inactive for an extended period. In such cases, the customer service representative will no longer follow up with the customer, resulting in the CC ID being changed to 0.

In [56]:
# operator id -1 means when system change a customer's cc to 0
cc_change_system = cc_change[cc_change['operator_id']==-1]
cc_change_system

,user_id,operator_id,prev_cc_id,trans_to_cc_id,trans_time
48694,158413,-1,167,0,1502382091
48695,158427,-1,179,0,1502382091
48696,158407,-1,179,0,1502382091
48697,158404,-1,115,0,1502382091
48698,158405,-1,179,0,1502382091
...,...,...,...,...,...
1675698,16482220,-1,235,0,1690394139
1675699,16575395,-1,235,0,1690394206
1675700,16581832,-1,647,0,1690394206
1675701,16582005,-1,167,0,1690394206


In [57]:
# Get the userid list of customers whose ccid was changed to 0
member_null_cc = customer_reach[customer_reach['cc_id']==0]
member_list_null_cc = member_null_cc['user_id'].to_list()

# Extract the cc_change record for members whose cc is 0
member_null_cc_change = cc_change_system[cc_change_system['user_id'].isin(member_list_null_cc)]
member_null_cc_change

,user_id,operator_id,prev_cc_id,trans_to_cc_id,trans_time
48694,158413,-1,167,0,1502382091
48695,158427,-1,179,0,1502382091
48696,158407,-1,179,0,1502382091
48697,158404,-1,115,0,1502382091
48698,158405,-1,179,0,1502382091
...,...,...,...,...,...
1675578,141995,-1,18,0,1690307720
1675696,16575386,-1,167,0,1690394139
1675697,16571347,-1,134,0,1690394139
1675698,16482220,-1,235,0,1690394139


In [58]:
member_null_cc_change['user_id'].value_counts()

155715      25
157101      21
157248      20
157060      20
157077      19
            ..
151909       1
151928       1
151929       1
151930       1
16579365     1
Name: user_id, Length: 113063, dtype: int64

Many customers have multiple CC change records. This occurs when a customer is transferred from one CC to another, usually when the original CC finds it challenging to continue providing follow-up. Prior to assigning a new CC to the customer, the customer's CC ID is temporarily changed to 0.



For analysis purposes, only the most recent CC responsible for the customer will be considered. This is because they are the CC present when the customer makes their final decisions.

In [59]:
# Sort the 'cc_change' table in descending order based on the 'transtime' column
member_null_cc_change_sorted = member_null_cc_change.sort_values(by='trans_time', ascending=False)

# Drop duplicate rows based on the 'studentid' column, keeping only the first occurrence
# This will ensure that only the most recent cc is retained for each user
member_null_cc_change_unique = member_null_cc_change_sorted.drop_duplicates(subset='user_id', keep='first')

# Extract the user_id and cc_id from the table in order to merge into customer_reach table
cc_prev = member_null_cc_change_unique[['user_id', 'prev_cc_id']].copy()
cc_prev.rename(columns={'prev_cc_id': 'cc_id'}, inplace=True)

cc_prev

,user_id,cc_id
1675699,16575395,235
1675698,16482220,235
1675697,16571347,134
1675696,16575386,167
1675574,16578910,490
...,...,...
49218,158065,154
48930,158385,200
48925,158529,200
48768,158437,200


In [60]:
# Extract the user_id originally has cc_id in the table
member_with_cc = customer_reach[customer_reach['cc_id']!=0]
member_with_cc = member_with_cc[['user_id', 'cc_id']].copy()
member_with_cc

,user_id,cc_id
5,16532204,100.0
6,16398016,197.0
16417,132355,46.0
16418,133975,46.0
16420,164996,47.0
...,...,...
237744,16558015,647.0
237746,16569340,647.0
237747,16578239,647.0
237748,16578345,647.0


In [61]:
# Concatenate two tables cc_prev and member_with_cc
cc_complete = pd.concat([cc_prev, member_with_cc])

# Drop the incomplete cc_id list from the customer_reach table
customer_reach = customer_reach.drop('cc_id',axis=1)

# Merge the complete cc list into the customer_reach table
customer_reach = pd.merge(customer_reach, cc_complete, on='user_id', how='left')
customer_reach

,user_id,channel,cc_id
0,16410781,1663.0,NaN
1,16448486,288.0,100.0
2,16531800,3009.0,650.0
3,16532204,288.0,100.0
4,16398016,672.0,197.0
...,...,...,...
158085,16558015,3109.0,647.0
158086,16569340,3156.0,647.0
158087,16578239,3156.0,647.0
158088,16578345,3156.0,647.0


In [62]:
customer_reach['cc_id'].replace(np.nan, 0, inplace=True)

#### Feature 2: num_cc

In [63]:
member_cc_change = cc_change[cc_change['user_id'].isin(member_analysis_list)]
member_cc_change

,user_id,operator_id,prev_cc_id,trans_to_cc_id,trans_time
0,161710,145,111,143,1502356636
1,161709,204,0,204,1502357188
2,161716,204,212,205,1502357254
3,161717,204,212,204,1502357282
18598,131638,-2,0,0,0
...,...,...,...,...,...
1675697,16571347,-1,134,0,1690394139
1675698,16482220,-1,235,0,1690394139
1675699,16575395,-1,235,0,1690394206
1675742,16553658,134,0,134,1690447980


In [64]:
member_cc_change['prev_cc_id'].replace(0, np.nan, inplace=True)
member_cc_change['trans_to_cc_id'].replace(0, np.nan, inplace=True)
member_cc_change

/var/folders/b9/g48b_66n63n3rwlbb7dhxm0r0000gn/T/ipykernel_38401/2840716736.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  member_cc_change['prev_cc_id'].replace(0, np.nan, inplace=True)
/var/folders/b9/g48b_66n63n3rwlbb7dhxm0r0000gn/T/ipykernel_38401/2840716736.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  member_cc_change['trans_to_cc_id'].replace(0, np.nan, inplace=True)


,user_id,operator_id,prev_cc_id,trans_to_cc_id,trans_time
0,161710,145,111.0,143.0,1502356636
1,161709,204,NaN,204.0,1502357188
2,161716,204,212.0,205.0,1502357254
3,161717,204,212.0,204.0,1502357282
18598,131638,-2,NaN,NaN,0
...,...,...,...,...,...
1675697,16571347,-1,134.0,NaN,1690394139
1675698,16482220,-1,235.0,NaN,1690394139
1675699,16575395,-1,235.0,NaN,1690394206
1675742,16553658,134,NaN,134.0,1690447980


In [65]:
# Combine 'prevccid' and 'transtoccid' columns into a single column
member_cc_change['combined_cc'] = member_cc_change[['prev_cc_id', 'trans_to_cc_id']].values.tolist()

# Group 'customer_cc_change' by 'userid' and create a list of unique values in 'combined_cc' column
cc_change_lists = member_cc_change.groupby('user_id')['combined_cc'].apply(lambda x: list(set([item for sublist in x for item in sublist if pd.notnull(item)]))).reset_index()

# Calculate the length of each list in 'combined_cc' column and store it in 'num_cc' column
cc_change_lists['num_cc'] = cc_change_lists['combined_cc'].apply(lambda x: len(x))

# Merge 'customer_conversion' with 'cc_change_lists' based on 'userid'
customer_reach = customer_reach.merge(cc_change_lists[['user_id', 'num_cc']], on='user_id', how='left')

customer_reach['num_cc'].replace(np.nan, 0, inplace=True)
customer_reach

/var/folders/b9/g48b_66n63n3rwlbb7dhxm0r0000gn/T/ipykernel_38401/1620527678.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  member_cc_change['combined_cc'] = member_cc_change[['prev_cc_id', 'trans_to_cc_id']].values.tolist()


,user_id,channel,cc_id,num_cc
0,16410781,1663.0,0.0,0.0
1,16448486,288.0,100.0,4.0
2,16531800,3009.0,650.0,2.0
3,16532204,288.0,100.0,2.0
4,16398016,672.0,197.0,0.0
...,...,...,...,...
158085,16558015,3109.0,647.0,3.0
158086,16569340,3156.0,647.0,0.0
158087,16578239,3156.0,647.0,0.0
158088,16578345,3156.0,647.0,0.0


#### Feature 3: channel

There are two levels of classification for channel. The first level is more general, and the second level is more detailed. The database directly stored second-level channel in the member_detail table. However, the granularity of second-level is too small, as there are more than 200 channels, so the first_level channel data is prefered for analysis. Therefore, to find first-level channel data, requires some data processing through the data key tables. 

In [66]:
customer_reach['channel'].value_counts()

2205.0    15287
298.0     14810
296.0     13583
791.0      9735
1751.0     8406
          ...  
168.0         1
286.0         1
699.0         1
394.0         1
1886.0        1
Name: channel, Length: 280, dtype: int64

In [67]:
# Find the attribute name for channel
channel_list = customer_reach['channel'].unique().tolist()
attribute_channel = attribute[attribute['attri_id'].isin(channel_list)]
attribute_channel

,attri_id,attri_name,attri_name_en,pid
142,168,我趣旅行,None,90.0
209,277,网易有道,None,95.0
211,280,官网400呼入,None,93.0
213,282,淘宝呼入,None,93.0
214,283,自动外呼,None,93.0
...,...,...,...,...
2729,3164,2023 天猫直播,None,99.0
2735,3170,2023海贝在线英语,None,99.0
2738,3173,抖音宝妈博主合作(KOL),None,3172.0
2739,3174,抖音宝妈博主推荐,None,3172.0


In [68]:
attribute_channel = attribute_channel.rename(columns={'attri_id': 'channel'})
customer_reach = customer_reach.merge(attribute_channel[['channel', 'pid']], on='channel', how='left')
customer_reach

,user_id,channel,cc_id,num_cc,pid
0,16410781,1663.0,0.0,0.0,1662.0
1,16448486,288.0,100.0,4.0,97.0
2,16531800,3009.0,650.0,2.0,2045.0
3,16532204,288.0,100.0,2.0,97.0
4,16398016,672.0,197.0,0.0,510.0
...,...,...,...,...,...
158085,16558015,3109.0,647.0,3.0,1662.0
158086,16569340,3156.0,647.0,0.0,1662.0
158087,16578239,3156.0,647.0,0.0,1662.0
158088,16578345,3156.0,647.0,0.0,1662.0


In [69]:
customer_reach.drop('channel',axis=1,inplace=True)
customer_reach = customer_reach.rename(columns={'pid': 'channel'})
customer_reach['channel'].replace(np.nan, 0, inplace=True)

#### Feature 4: trial_money

In [70]:
# Get the product type list
product_list = order[['product_id', 'product_name']].drop_duplicates()

# Sort the values in ascending order
product_list = product_list.sort_values('product_id')

product_list

,product_id,product_name
144789,0.0,None
2,17.0,海螺币
19447,17.0,体验卡
0,21.0,菲律宾外教
1834,21.0,菲律宾外教次卡
1776,23.0,专业中教
1838,23.0,专业中教次卡
1862,24.0,测评卡
1,25.0,欧美外教次卡
106075,25.0,欧美外交次卡


In [71]:
# According to DBeaver data filtering, 17 is for 海螺币, 26 is for 体验卡
# 17 for 体验卡 is outlier
product_outlier = order[(order['product_id'] == 17) & (order['product_name'] == '体验卡')]
product_outlier

,user_id,product_id,product_name,buy_type,product_money
19447,141080.0,17.0,体验卡,690.0,0.0
19450,141081.0,17.0,体验卡,690.0,0.0
19451,141082.0,17.0,体验卡,690.0,0.0
19452,141084.0,17.0,体验卡,690.0,0.0


In [72]:
# Check the buy_type of the outliers
attribute_buy_type_outlier = attribute[attribute['attri_id']==690]
attribute_buy_type_outlier

,attri_id,attri_name,attri_name_en,pid
563,690,受邀注册赠送体验卡,None,739.0


In [73]:
# Get the orders of all the trial classes
trial_order = order[order['product_id']==26]

# Add the free trial (product id = 17) to the table
trial_order = pd.concat([trial_order,product_outlier],axis=0)
trial_order

,user_id,product_id,product_name,buy_type,product_money
2129,119379.0,26.0,体验卡,104.0,0.0
2131,119241.0,26.0,体验卡,389.0,0.0
2134,119439.0,26.0,体验卡,104.0,39.0
2135,119439.0,26.0,体验卡,104.0,39.0
2136,119441.0,26.0,体验卡,104.0,39.0
...,...,...,...,...,...
231208,16583135.0,26.0,体验卡,749.0,0.0
19447,141080.0,17.0,体验卡,690.0,0.0
19450,141081.0,17.0,体验卡,690.0,0.0
19451,141082.0,17.0,体验卡,690.0,0.0


In [74]:
# Filter the trial order by member_analysis_list
trial_order = trial_order[trial_order['user_id'].isin(member_analysis_list)]

In [75]:
# Group 'trial_order' by 'userid' and calculate the mean of 'product_money'
trial_money = trial_order.groupby('user_id')['product_money'].sum().reset_index()

# Merge 'customer_conversion_2' with 'trial_money' based on 'userid'
customer_reach = customer_reach.merge(trial_money, on='user_id', how='left')
customer_reach

,user_id,cc_id,num_cc,channel,product_money
0,16410781,0.0,0.0,1662.0,NaN
1,16448486,100.0,4.0,97.0,0.0
2,16531800,650.0,2.0,2045.0,0.0
3,16532204,100.0,2.0,97.0,0.0
4,16398016,197.0,0.0,510.0,NaN
...,...,...,...,...,...
158085,16558015,647.0,3.0,1662.0,0.0
158086,16569340,647.0,0.0,1662.0,0.0
158087,16578239,647.0,0.0,1662.0,NaN
158088,16578345,647.0,0.0,1662.0,NaN


In [76]:
customer_reach['product_money'].replace(np.nan, 0, inplace=True)
customer_reach = customer_reach.rename(columns={'product_money': 'trial_money'})

#### Dependent Variable: trial_completed

In [77]:
# Find the class that are paid/reserved by trial-class card in the account
trial_class_reservation = class_reservation[class_reservation['pay_type']==26]

# Find the trial class reservations that are completed successfully
trial_class_completed = trial_class_reservation[trial_class_reservation['class_order_status']==2]
trial_class_completed

,class_order_id,user_id,class_order_status,pay_type
7773,7774,119379,2,26
7792,7793,119443,2,26
7804,7805,119445,2,26
7849,7850,119449,2,26
7855,7856,119447,2,26
...,...,...,...,...
1359220,1363404,16583078,2,26
1359222,1363406,16583073,2,26
1359238,1363422,16582687,2,26
1359245,1363429,16582825,2,26


In [78]:
trial_user_list = trial_class_completed['user_id'].unique().tolist()
customer_reach['trial_completed'] = customer_reach['user_id'].apply(lambda x: 1 if x in trial_user_list else 0)
customer_reach

,user_id,cc_id,num_cc,channel,trial_money,trial_completed
0,16410781,0.0,0.0,1662.0,0.0,0
1,16448486,100.0,4.0,97.0,0.0,1
2,16531800,650.0,2.0,2045.0,0.0,1
3,16532204,100.0,2.0,97.0,0.0,1
4,16398016,197.0,0.0,510.0,0.0,0
...,...,...,...,...,...,...
158085,16558015,647.0,3.0,1662.0,0.0,0
158086,16569340,647.0,0.0,1662.0,0.0,0
158087,16578239,647.0,0.0,1662.0,0.0,0
158088,16578345,647.0,0.0,1662.0,0.0,0


In [79]:
customer_reach['trial_completed'].value_counts()

0    98513
1    59577
Name: trial_completed, dtype: int64

# Finalization

In [80]:
customer_reach.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 158090 entries, 0 to 158089
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   user_id          158090 non-null  int64  
 1   cc_id            158090 non-null  float64
 2   num_cc           158090 non-null  float64
 3   channel          158090 non-null  float64
 4   trial_money      158090 non-null  float64
 5   trial_completed  158090 non-null  int64  
dtypes: float64(4), int64(2)
memory usage: 8.4 MB


In [81]:
# Save the DataFrame to a CSV file
customer_reach.to_csv('customer_reach.csv', index=False)

print("saved to CSV file successfully.")

saved to CSV file successfully.
